In [1]:
%reload_ext autoreload
%autoreload 2
 
from matplotlib import pyplot as plt
%matplotlib inline
 
import tensorflow as tf
from tf_keras_vis.utils import num_of_gpus
 
_, gpus = num_of_gpus()
print('Tensorflow recognized {} GPUs'.format(gpus))

2024-05-10 21:52:05.379138: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 21:52:05.379185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 21:52:05.380148: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-10 21:52:05.385983: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 21:52:06.036914: W tensorflow/comp

Tensorflow recognized 0 GPUs


2024-05-10 21:52:06.862628: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-05-10 21:52:06.862703: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: 0ff20dead65f
2024-05-10 21:52:06.862738: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: 0ff20dead65f
2024-05-10 21:52:06.862909: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 550.54.15
2024-05-10 21:52:06.862963: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 550.54.15
2024-05-10 21:52:06.862984: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 550.54.15


In [2]:
import pennylane as qml
from pennylane import numpy as np
from tensorflow import keras

mnist_dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

n_epochs = 30   # Number of optimization epochs
n_layers = 1    # Number of random layers
n_train = 80    # Size of the train dataset
n_test = 30     # Size of the test dataset
n_batches = 4     # Size of the batches

# Reduce dataset size
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]

# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255

# Add extra dimension for convolution channels
train_images = np.array(train_images[..., tf.newaxis], requires_grad=False)
test_images = np.array(test_images[..., tf.newaxis], requires_grad=False)

n_qubits = 4

dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface='tf')
def qnotnode(inputs):
    inputs *= np.pi
    # Encoding of 4 classical input values
    qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')

    # Filter from arxiv.org/abs/2308.14930

    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[0, 3])


    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(n_qubits)]

class ConvQLayer(qml.qnn.KerasLayer):
    
    def call(self, inputs):

        out = tf.Variable(tf.zeros((n_batches, 14, 14, n_qubits)))
        for b in range(n_batches):
            # Loop over the coordinates of the top-left pixel of 2X2 squares
            for j in range(0, 28, 2):
                for k in range(0, 28, 2):
                    # Process a squared 2x2 region of the image with a quantum circuit
                    q_results = tf.stack(
                        [
                            inputs[b,j, k, 0],
                            inputs[b,j, k + 1, 0],
                            inputs[b,j + 1, k, 0],
                            inputs[b,j + 1, k + 1, 0]
                        ],
                        axis = 1
                    )
                    q_results = super().call(q_results)
                    # Assign expectation values to different channels of the output pixel (j/2, k/2)
                    for c in range(n_qubits):
                        out[b,j // 2, k // 2, c] = q_results[c]
        return out 

qnotlayer = qml.qnn.KerasLayer(qnotnode, {}, output_dim=[28,28,n_qubits])

qnotlayer.trainable = False

def Qnot_Model():
    """Initializes and returns a custom Keras model
    which is ready to be trained."""
    model = keras.models.Sequential([
        qnotlayer,
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")
    ])
    model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

qnot_model = Qnot_Model()

qnot_history = qnot_model.fit(
    train_images,
    train_labels,
    validation_data=(test_images, test_labels),
    batch_size = n_batches,
    epochs=n_epochs,
    verbose=2
)

Epoch 1/30
20/20 - 2s - loss: 3.0419 - accuracy: 0.2750 - val_loss: 2.4549 - val_accuracy: 0.3333 - 2s/epoch - 86ms/step
Epoch 2/30
20/20 - 2s - loss: 1.5368 - accuracy: 0.4625 - val_loss: 1.2140 - val_accuracy: 0.6667 - 2s/epoch - 78ms/step
Epoch 3/30
20/20 - 2s - loss: 0.7850 - accuracy: 0.7375 - val_loss: 1.4766 - val_accuracy: 0.6333 - 2s/epoch - 76ms/step
Epoch 4/30
20/20 - 2s - loss: 0.5034 - accuracy: 0.8875 - val_loss: 0.9045 - val_accuracy: 0.6667 - 2s/epoch - 79ms/step
Epoch 5/30
20/20 - 2s - loss: 0.3459 - accuracy: 0.9375 - val_loss: 1.0090 - val_accuracy: 0.7667 - 2s/epoch - 75ms/step
Epoch 6/30
20/20 - 1s - loss: 0.3252 - accuracy: 0.9125 - val_loss: 1.0446 - val_accuracy: 0.6667 - 1s/epoch - 72ms/step
Epoch 7/30
20/20 - 2s - loss: 0.2414 - accuracy: 0.9375 - val_loss: 0.9025 - val_accuracy: 0.7333 - 2s/epoch - 76ms/step
Epoch 8/30
20/20 - 1s - loss: 0.1628 - accuracy: 0.9625 - val_loss: 0.8812 - val_accuracy: 0.7333 - 1s/epoch - 71ms/step
Epoch 9/30
20/20 - 1s - loss: 0.

In [3]:
model = qnot_model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 28, 28, 4)         0         
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 10)                31370     
                                                                 
Total params: 31370 (122.54 KB)
Trainable params: 31370 (122.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
model.save('model')

Cause: Unable to locate the source code of <function _gcd_import at 0x7999938d7d80>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function _gcd_import at 0x7999938d7d80>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @t

INFO:tensorflow:Assets written to: model/assets


In [9]:
load = keras.models.load_model('model')

In [10]:
load.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 28, 28, 4)         0         
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 10)                31370     
                                                                 
Total params: 31370 (122.54 KB)
Trainable params: 31370 (122.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model(train_images)

<tf.Tensor: shape=(80, 10), dtype=float32, numpy=
array([[1.20761991e-03, 3.74046667e-03, 1.63640396e-03, 1.00690871e-02,
        1.57830436e-04, 9.74630535e-01, 2.90802657e-03, 1.49816988e-04,
        8.48014082e-04, 4.65216162e-03],
       [9.98854995e-01, 1.51292425e-05, 2.95401769e-05, 2.14092543e-05,
        1.99213919e-05, 6.95564027e-04, 1.17945106e-04, 1.55700909e-04,
        8.39427739e-05, 5.66017752e-06],
       [1.44867863e-05, 6.04510433e-06, 3.53058203e-05, 4.22612386e-04,
        9.99143243e-01, 6.27362751e-05, 2.12458690e-04, 1.89848215e-05,
        4.77247204e-06, 7.92992796e-05],
       [4.15370960e-05, 9.88576651e-01, 4.49136598e-04, 1.67416380e-04,
        2.69370852e-03, 1.17553724e-03, 3.42138192e-05, 2.83637521e-04,
        6.50315871e-03, 7.50219697e-05],
       [9.51909169e-05, 6.65323809e-04, 2.47278775e-04, 9.34424825e-05,
        6.02936186e-03, 1.19991681e-04, 5.59484935e-04, 4.96992003e-03,
        3.02921608e-03, 9.84190762e-01],
       [1.00870919e-03, 4